# Part I. Building an ETL Pipeline to Pre-Processing Files

#### Import Python packages 

In [1]:
# Here all required packages are imported
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Return current working directory
# print(os.getcwd())

#  Event data is stored in the current folder and subfolder, that data is stored in the filepath variable 
filepath = os.getcwd() + '/event_data'

# Now we will loop through the files in the filepath variable to create a list of files and get each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# Loop through every filepath in file_path_list to open a csv and append each line of the csv to full_data_rows_list
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)     
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full to insert data into thApache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Get the number of rows in the event_datafile_new.csv
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Getting the Data into Cassandra  

#### Creating a Cluster

In [5]:
df = pd.read_csv('event_datafile_new.csv')

In [6]:
# Make a connection to a Cassandra instance from local machine 
# (127.0.0.1)
import cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a keyspace called sparkify if it doesn't already exist
session.execute(
    """CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [8]:
# Set the keyspace
session.set_keyspace('sparkify')

## Create queries to ask the following three questions of the data

### Query # 1
#### 1. Get the artist, song title, and song length from the music app history where the itemInSession  = 4, and sessionId = 338.

In [9]:
# Create a new table called music_by_session_id_and_item if it doesn't already exist
session.execute("""
CREATE TABLE IF NOT EXISTS music_by_session_id_and_item 
(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))
""")

                    

In [10]:
# Assign the event_datafile_new.csv to the file variable, loop through the csv
# and insert data from each row into the music_by_session_id_and_item table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO music_by_session_id_and_item (session_id, item_in_session, artist_name, song_title, song_length)"""
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verify that the data was inserted into the music_by_session_id_and_item table

##### The music_by_session_id_and_item table uses session_id as its partition key and item_in_session as its clustering column. This allows us to query the table by session_id and will ensure we receive songs in the order in which they were played.

In [11]:
# Select data and loop through to print results
rows = session.execute("""SELECT artist_name, song_title, song_length FROM music_by_session_id_and_item WHERE session_id = 338 AND item_in_session = 4;""")
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query #2
#### 2. Get the name of the artist, the song (sorted by itemInSession), and user's first and last name for userid = 10, sessionid = 182

In [12]:
# Create a new table called music_by_user_id_and_session_id if it doesn't already exist
session.execute("""
CREATE TABLE IF NOT EXISTS music_by_user_id_and_session_id
(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))
""")

                    

In [13]:
# Assign the event_datafile_new.csv to the file variable, loop through the csv
# and insert data from each row into the music_by_user_id_and_session_id
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = """INSERT INTO music_by_user_id_and_session_id (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

##### The music_by_user_id_and_session_id table uses (user_id, session_id) as it's composite key so that data is partitioned by user_id and session_id. This ensures that we can query the data by user_id and session_id. Since we have used item_in_session as the clustering column the songs will be returned in the order they were played.

In [14]:
# Select data and loop through to print results
rows_user_session = session.execute("""SELECT artist_name, song_title, user_first_name, user_last_name FROM music_by_user_id_and_session_id WHERE user_id = 10 AND session_id = 182;""")
for row in rows_user_session:
    print(row.artist_name, row.song_title, row.user_first_name, row.user_last_name)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query #3
#### 3. Get the user name (first and last) in music app history who listened to the song 'All Hands Against His Own'

In [15]:
# Create a new table called user_by_song if it doesn't already exist
session.execute("""
CREATE TABLE IF NOT EXISTS user_by_song
(song_title text, user_id int, session_id int, item_in_session int, artist_name text, user_first_name text, user_last_name text, PRIMARY KEY (song_title, user_id))
""")

In [16]:
# Assign the event_datafile_new.csv to the file variable, loop through the csv
# and insert data from each row into the user_by_song
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = """INSERT INTO user_by_song (song_title, user_id, session_id, item_in_session, artist_name, user_first_name, user_last_name)"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4]))

##### The user_by_song table uses song_title as the partition key and user_id as the clustering column. This allows us to query on song_title and have the results ordered by user_id. 

In [17]:
# Select data and loop through to print results
rows_user_session = session.execute("""SELECT user_first_name, user_last_name FROM user_by_song WHERE song_title = 'All Hands Against His Own';""")
for row in rows_user_session:
    print(row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables 

In [18]:
session.execute("""DROP TABLE music_by_session_id_and_item;""") 
session.execute("""DROP TABLE music_by_user_id_and_session_id;""")
session.execute("""DROP TABLE user_by_song;""")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()